# Live Data Pull from Twitter, Feb 2019

In [ ]:
print("Hello World! This was created on Feb 22, 2019. Update on 2/25/2019")

Here is an outline of the Twitter live tweet module:

* Notification given by sound and text on console that a tweet has occured;
* Notification provides timestamp and current tweet count; optional function for count beyond last mark.

__A simple notification tool whenever a tweet occurs.__

Note: class instantiated.

Instantiate class with bar = TwitterPull("tweethandle")

Preliminaries:

In [ ]:
import time
from datetime import datetime
import tweepy
from pygame import mixer
from gtts import gTTS

auth = tweepy.OAuthHandler(consumer_key, consumer_secret) #needs to be set.
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

Pygame mixer offers a playing function; gtts is google text-to-speech. All can be installed with pip install.

Similar to historical tweet pull.

In [ ]:
class TwitterPull:
    def __init__(self, handle = "RealDonaldTrump", first = 0):
        self._handle = handle
        self._a = None
        self._b = None
        self._stat = None
        self._first = first
        
    def pull(self):
        """Function to refresh and check for updates; get_user refreshes data.
        If user tweets, then the counter updates and plays a sound notification."""
        read = []
        User = api.get_user(self._handle) 
        if User.statuses_count == self._stat:
            pass
        else:
            diff = User.statuses_count - self._stat
            self._stat = User.statuses_count
            text = "New tweet from " + self._handle + "! Currently at " + str(self._stat - self._first)
            print(self._timerefresh(), text)
            tts = gTTS(text)
            tts.save("RDTs.mp3")
            self.play("RDTs.mp3")

            for tweet in tweepy.Cursor(api.user_timeline, id="realdonaldtrump", tweet_mode = "extended").items(diff):
                read.append(tweet)
            read.reverse()
            tts = gTTS("".join(read))
            tts.save("Trumptext.mp3")
            self.play("Trumptext.mp3")
            

__Something interesting here: If multiple tweets come at once, then they are read in order together. This is a good way to practice array queues or other queue implementations. Otherwise tweets come one by one and are read one by one.__

Relies on three basic functions: Sleep, Play (sound), and Time refresh

_Could_ split it up into another call so it's less wordy.


In [ ]:
    def sleep(self, t = 20):
        """Sleep t seconds."""
        time.sleep(t)
    
    def play(self, file):
            mixer.init()
            mixer.music.load(file)
            mixer.music.play()
    
    def _timerefresh(self):
        """Refresh the current time, Return a timestamp string: day/hour/min
        Ex:
            '02/05 19:49'
        """
        timestamp = datetime.now().strftime('%m/%d %H:%M')
        return timestamp

### The function to pull data continuously.

In [ ]:
    def continuouspull(self, n = 60*12):
        #default: half a day
        User = api.get_user("RealDonaldTrump"); self._stat = User.statuses_count

        print(str(User.statuses_count) + " currently.")

        for i in range(n):
            try:
                self.pull()
            except:
                input("pull failed.." + str(self._timerefresh()))
            self.sleep(20)
